In [1]:

import simpy
import random


In [43]:

# Maximo de vehiculos que puede recibir el negocio
MAX_VEHICULOS = 57 
# Total de maquinas de lavado con que cuenta el negocio
NUM_MAQUINAS = 3
# Tiempo que tarda en lavarse un vehiculo (minutos)
TIEMPO_LAVADO = 7 
# Intervalo de tiempo en que llegan vehiculos (minutos)
INTERVALO_LLEGADA = 9
# Tiempo de simulación
TIEMPO_SIMULACION = 23

class Lavanderia(object):
    
    def __init__(self, environment, num_maquinas, tiempo_lavado):
        # Guardamos como variable el entorno de ejecucion
        self.env=environment
        # Creamos el recurso que representa las maquinas
        self.maquinas = simpy.Resource(environment, num_maquinas)
        # Variable para el tiempo de lavado
        self.tiempo_lavado = tiempo_lavado
        
    def lavar_vehiculo(self, vehiculo):
        # Este metodo representa el proceso de lavado del vehículo.
        # Se ingresa el vehículo y se lava
        
        # Simulamos el tiempo que tarda en lavarse el vehiculo
        # Es importante notar que la instruccion "yield" es distinta de "sleep"
        # ya que esta ultima bloquea el hilo de ejecucion durante 't' unidades de tiempo,
        # mientras que 'yield' no bloquea el hilo de ejecucion, solo lo suspende mientras
        # el evento de 'lavado' se realice
        yield self.env.timeout(TIEMPO_LAVADO)
        
        # Simulamos que se ha limpiado parte (%) de la suciedad del vehiculo
        # Para el % generamos un entero entre 30 y 90
        print('Removido  {%d%%} suciedad vehiculo => %s ' \
              % (random.randint(30,90), vehiculo))
        
    
def llegada_vehiculo(env, nombre, lavanderia, num_maquina):
    # Usamos el reloj de la simulacion (env.now()) para indicar a la
    # hora que llega el vehiculo con el nombre pasado como parametro
    print('--> Llega vehiculo: %s a la hora %.2f.' % (nombre, env.now))
    
    # Especificamos que vamos a usar un recurso (Resource) que representa
    # la maquina de lavado
    with lavanderia.maquinas.request() as maquina:
        # Ocupamos la maquina de lavado
        yield maquina
        
        print('Vehiculo %s asignado a la maquina %s '%(nombre, num_maquina))
        # Indicamos que vehiculo entra a la lavanderia
        print('Entra vehiculo a lavarse: %s a la hora %.2f.' % (nombre, env.now))
        # Procesamos la operacion de lavado
        yield env.process(lavanderia.lavar_vehiculo(nombre))
        # Una vez que termina la llamada con 'yield', se indica que se ha lavado el vehiculo
        print('Vehiculo [%s] lavado a las %.2f.' % (nombre, env.now))
    
    
def ejecutar_simulacion(env, num_maquinas, tiempo_lavado, intervalo):
    lavanderia=Lavanderia(env, num_maquinas, tiempo_lavado)
    # Creamos 5 llegadas de vehiculos iniciales
    num_maquina=0
    for i in range(5):
        num_maquina +=1
        env.process(llegada_vehiculo(env, 'Vehiculo-%d'%(i+1),lavanderia,num_maquina))
        if(num_maquina ==num_maquinas): num_maquina=0
    # Ejecutamos la simulacion
    num_maquina=0
    while True:
        yield env.timeout(random.randint(intervalo-2, intervalo+2))
        i+=1
        num_maquina +=1
        # Mientras se lavan los vehiculos generamos mas vehiculos
        env.process(llegada_vehiculo(env,'Vehiculo-%d'%(i+1),lavanderia, num_maquina))
        
        if(num_maquina ==num_maquinas): num_maquina=0
print('Lavanderia Javier Vazquez')
# Inicializamos la semilla aleatoria
random.seed(77)

# Creamos el entorno de simulacion
env=simpy.Environment()
env.process(ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA))

# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = TIEMPO_SIMULACION)

Lavanderia Javier Vazquez
--> Llega vehiculo: Vehiculo-1 a la hora 0.00.
--> Llega vehiculo: Vehiculo-2 a la hora 0.00.
--> Llega vehiculo: Vehiculo-3 a la hora 0.00.
--> Llega vehiculo: Vehiculo-4 a la hora 0.00.
--> Llega vehiculo: Vehiculo-5 a la hora 0.00.
Vehiculo Vehiculo-1 asignado a la maquina 1 
Entra vehiculo a lavarse: Vehiculo-1 a la hora 0.00.
Vehiculo Vehiculo-2 asignado a la maquina 2 
Entra vehiculo a lavarse: Vehiculo-2 a la hora 0.00.
Vehiculo Vehiculo-3 asignado a la maquina 3 
Entra vehiculo a lavarse: Vehiculo-3 a la hora 0.00.
Removido  {50%} suciedad vehiculo => Vehiculo-1 
Removido  {42%} suciedad vehiculo => Vehiculo-2 
Removido  {45%} suciedad vehiculo => Vehiculo-3 
Vehiculo [Vehiculo-1] lavado a las 7.00.
Vehiculo [Vehiculo-2] lavado a las 7.00.
Vehiculo [Vehiculo-3] lavado a las 7.00.
Vehiculo Vehiculo-4 asignado a la maquina 1 
Entra vehiculo a lavarse: Vehiculo-4 a la hora 7.00.
Vehiculo Vehiculo-5 asignado a la maquina 2 
Entra vehiculo a lavarse: Vehicu

#### Tiempo transporte a las maquinas 

In [47]:
# Maximo de vehiculos que puede recibir el negocio
MAX_VEHICULOS = 57 
# Total de maquinas de lavado con que cuenta el negocio
NUM_MAQUINAS = 3
# Tiempo que tarda en lavarse un vehiculo (minutos)
TIEMPO_LAVADO = 7 
# Intervalo de tiempo en que llegan vehiculos (minutos)
INTERVALO_LLEGADA = 9
# Tiempo de simulación
TIEMPO_SIMULACION = 30

TIEMPO_TRANSPORTE=2
TIEMPO_SALIDA = 3
class Lavanderia(object):
    
    def __init__(self, environment, num_maquinas, tiempo_lavado):
        # Guardamos como variable el entorno de ejecucion
        self.env=environment
        # Creamos el recurso que representa las maquinas
        self.maquinas = simpy.Resource(environment, num_maquinas)
        # Variable para el tiempo de lavado
        self.tiempo_lavado = tiempo_lavado
        
    def lavar_vehiculo(self, vehiculo):
        # Este metodo representa el proceso de lavado del vehículo.
        # Se ingresa el vehículo y se lava
        
        # Simulamos el tiempo que tarda en lavarse el vehiculo
        # Es importante notar que la instruccion "yield" es distinta de "sleep"
        # ya que esta ultima bloquea el hilo de ejecucion durante 't' unidades de tiempo,
        # mientras que 'yield' no bloquea el hilo de ejecucion, solo lo suspende mientras
        # el evento de 'lavado' se realice
        yield self.env.timeout(TIEMPO_LAVADO)
        
        # Simulamos que se ha limpiado parte (%) de la suciedad del vehiculo
        # Para el % generamos un entero entre 30 y 90
        print('Removido  {%d%%} suciedad vehiculo => %s ' \
              % (random.randint(30,99), vehiculo))
        
    def llegada_vehiculo_maquina(self, tiempo):
        #simulacion tiempo en llegar vehiculo hasta la maquina 
        yield self.env.timeout(tiempo)
    
    def salida_vehiculo(self, tiempo):
        # simulacion tiempo salida vehiculo 
        yield self.env.timeout(tiempo
                              )
def llegada_vehiculo(env, nombre, lavanderia, num_maquina,tiempo_transporte, tiempo_salida):
    # Usamos el reloj de la simulacion (env.now()) para indicar a la
    # hora que llega el vehiculo con el nombre pasado como parametro
    print('---> Llega vehiculo: %s a la hora %.2f.' % (nombre, env.now))
    
    # Especificamos que vamos a usar un recurso (Resource) que representa
    # la maquina de lavado
    llega= env.now
    with lavanderia.maquinas.request() as maquina:
        # Ocupamos la maquina de lavado
        yield maquina
        pasa=env.now
        
        print('Se transporta:  %s a la hora %.2f. luego de esperar %s minutos'%(nombre, env.now,(pasa-llega)))
        yield env.process(lavanderia.llegada_vehiculo_maquina(tiempo_transporte))
        
        print('Vehiculo:  %s asignado a la maquina:  %s '%(nombre, num_maquina))
        # Indicamos que vehiculo entra a la lavanderia
        print('Entra vehiculo a lavarse: %s a la hora: %.2f.' % (nombre, env.now))
        # Procesamos la operacion de lavado
        yield env.process(lavanderia.lavar_vehiculo(nombre))
        # Una vez que termina la llamada con 'yield', se indica que se ha lavado el vehiculo
        print('Vehiculo:  [%s] lavado a las %.2f.' % (nombre, env.now))
        
        yield env.process(lavanderia.salida_vehiculo(tiempo_salida))
        print('<--- Vehiculo:  [%s] abandona las instalaciones a las %.2f '%(nombre, env.now))
    
def ejecutar_simulacion(env, num_maquinas, tiempo_lavado, intervalo, tiempo_transporte, tiempo_salida):
    lavanderia=Lavanderia(env, num_maquinas, tiempo_lavado)
    # Creamos 5 llegadas de vehiculos iniciales
    num_maquina=0
    for i in range(5):
        num_maquina +=1
        env.process(llegada_vehiculo(env, 'Vehiculo-%d'%(i+1),lavanderia,num_maquina,tiempo_transporte,tiempo_salida))
        if(num_maquina ==num_maquinas): num_maquina=0
    # Ejecutamos la simulacion
    num_maquina=0
    while True:
        yield env.timeout(random.randint(intervalo-2, intervalo+2))
        i+=1
        num_maquina +=1
        # Mientras se lavan los vehiculos generamos mas vehiculos
        env.process(llegada_vehiculo(env,'Vehiculo-%d'%(i+1),lavanderia, num_maquina,tiempo_transporte,tiempo_salida ))
        
        if(num_maquina ==num_maquinas): num_maquina=0
print('-'*5,'Lavanderia JAVIER VAZQUEZ','-'*5,'\n')
# Inicializamos la semilla aleatoria
random.seed(77)

# Creamos el entorno de simulacion
env=simpy.Environment()
env.process(ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA, TIEMPO_TRANSPORTE, TIEMPO_SALIDA))

# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = TIEMPO_SIMULACION)

----- Lavanderia JAVIER VAZQUEZ ----- 

---> Llega vehiculo: Vehiculo-1 a la hora 0.00.
---> Llega vehiculo: Vehiculo-2 a la hora 0.00.
---> Llega vehiculo: Vehiculo-3 a la hora 0.00.
---> Llega vehiculo: Vehiculo-4 a la hora 0.00.
---> Llega vehiculo: Vehiculo-5 a la hora 0.00.
Se transporta:  Vehiculo-1 a la hora 0.00. luego de esperar 0 minutos
Se transporta:  Vehiculo-2 a la hora 0.00. luego de esperar 0 minutos
Se transporta:  Vehiculo-3 a la hora 0.00. luego de esperar 0 minutos
Vehiculo:  Vehiculo-1 asignado a la maquina:  1 
Entra vehiculo a lavarse: Vehiculo-1 a la hora: 2.00.
Vehiculo:  Vehiculo-2 asignado a la maquina:  2 
Entra vehiculo a lavarse: Vehiculo-2 a la hora: 2.00.
Vehiculo:  Vehiculo-3 asignado a la maquina:  3 
Entra vehiculo a lavarse: Vehiculo-3 a la hora: 2.00.
---> Llega vehiculo: Vehiculo-6 a la hora 9.00.
Removido  {55%} suciedad vehiculo => Vehiculo-1 
Removido  {60%} suciedad vehiculo => Vehiculo-2 
Removido  {54%} suciedad vehiculo => Vehiculo-3 
Vehicu